In [2]:
import pathlib
import tarfile
import numpy as np

In [ ]:
for zipped_file in pathlib.Path('..\\data\\').glob('*.tar.gz'):
    tar = tarfile.open(zipped_file, 'r:gz')
    tar.extractall('..\\completeDataset')
    tar.close()

In [ ]:
print('done')

In [3]:
from IPython.display import HTML, display

def progress(value, max=100, message = ''):
    return HTML(f"""<progress
                    value="{value}"
                    max="{max}",
                    style="width: 50%"
                > {value}
                </progress> {message}""")

def message(value = ''):
    return HTML(f"{value}")

In [7]:
import json
from PIL import Image
import csv
import os

directory = '..\\completeDataset'
outDirectory = 'dataset'
if not os.path.exists(outDirectory):
          os.makedirs(outDirectory)
testCut = 0.1
count = 0
frameSize = (256,256)
with open(os.path.join(outDirectory,'train.csv'), mode='w') as trainDataframe, open(os.path.join(outDirectory,'test.csv'), mode='w') as testDataframe:
  trainDfWriter = csv.writer(trainDataframe, delimiter=';')
  testDfWriter = csv.writer(testDataframe, delimiter=';')
  trainDfWriter.writerow(['image', 'leftX', 'leftY', 'rightX', 'rightY', 'width', 'height'])
  testDfWriter.writerow(['image', 'leftX', 'leftY', 'rightX', 'rightY', 'width', 'height'])
  decompressCount = 0
  dirCount = len(os.listdir(directory))
  progressBar = display(message(''), display_id=True)
  for dirName in os.listdir(directory):
    progressBar.update(progress(decompressCount,
                                dirCount,
                                f'{decompressCount}/{dirCount} decompressed directory'))
    infoPath = os.path.join(directory, dirName, 'info.json')
    if not os.path.exists(infoPath):
      continue
    with open(infoPath) as infofile:
      info = json.load(infofile)
    frameNumber = info['TotalFrames']
    testCutPoint = frameNumber * (1 - testCut)
    leftPath = os.path.join(directory, dirName, 'appleLeftEye.json')
    rightPath = os.path.join(directory, dirName, 'appleRightEye.json')
    facePath = os.path.join(directory, dirName, 'appleFace.json')
    with open(leftPath) as l, open(rightPath) as r, open(facePath) as f:
      leftEyes = json.load(l)
      rightEyes = json.load(r)
      faces = json.load(f)
    processedFrames = 0
    for i in range(frameNumber):
      if (leftEyes['IsValid'][i] and rightEyes['IsValid'][i]):
        framePath = os.path.join(directory, dirName, 'frames', '%05d.jpg' % i)
        saveDirPath = os.path.join(outDirectory, 'frames')
        if not os.path.exists(saveDirPath):
          os.makedirs(saveDirPath)
        savePath = os.path.join(saveDirPath, '%07d.jpg' % count)
        with Image.open(framePath) as frame:
          width, height = frame.size
          fresizesFrame = frame.resize(frameSize)
          fresizesFrame.save(savePath, 'JPEG')
        leftX = (faces['X'][i]+leftEyes['X'][i]+(leftEyes["W"][i]/2))/width
        leftY = (faces['Y'][i]+leftEyes['Y'][i]+(leftEyes["H"][i]/2))/height
        rightX = (faces['X'][i]+rightEyes['X'][i]+(rightEyes["W"][i]/2))/width
        rightY = (faces['Y'][i]+rightEyes['Y'][i]+(rightEyes["H"][i]/2))/height
        if np.random.random() > testCut:
          trainDfWriter.writerow([savePath, leftX, leftY, rightX, rightY, width, height])
        else:
          testDfWriter.writerow([savePath, leftX, leftY, rightX, rightY, width, height])
        count += 1
        processedFrames += 1
        if processedFrames > 100:
          break
    decompressCount += 1
progressBar.update(message(f'loaded {count} frames'))